# <u>Test Exercice 4 - Endogeneity<u>

__<u>Jorge Pineño Pérez<u>__ / Module 5 - Endogeneity

A challenging and very relevant economic problem is the measurement of the returns to schooling. In this question we will use the following variables on 3010 US men:

- `logw`: log wage
- `educ`: number of years of schooling
- `age`: age of the individual in years
- `exper`: working experience in years
- `smsa`: summy indicating whether the individual lived in the south
- `nearc`: dummy indicating whether the individual lived near a 4-year college
- `dadeduc`: education of the individual's father (in years)
- `momeduc`: education of the individual's mother (in years)

This data is a selection of the data used by D. Card (1995)

In [22]:
# import packages
install.packages("readxl")
install.packages("dplyr")
install.packages("car")
install.packages("AER")
library(readxl)
library(dplyr)
library(car)
library(AER)

Warning message:
"package 'readxl' is in use and will not be installed"
Warning message:
"package 'dplyr' is in use and will not be installed"
Warning message:
"package 'car' is in use and will not be installed"
Installing package into 'C:/Users/USER/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)

also installing the dependency 'sandwich'




package 'sandwich' successfully unpacked and MD5 sums checked
package 'AER' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\USER\AppData\Local\Temp\RtmpmAQo52\downloaded_packages


Cargando paquete requerido: lmtest

Cargando paquete requerido: zoo


Adjuntando el paquete: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Cargando paquete requerido: sandwich

Cargando paquete requerido: survival



In [3]:
# import the dataframe
retsed <- read_excel('TestExer4_Wage-round1.xls')
head(retsed)

logw,educ,age,exper,smsa,south,nearc,daded,momed
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
6.306275,7,29,16,1,0,0,9.94,10.25
6.175867,12,27,9,1,0,0,8.00,8.00
6.580639,12,34,16,1,0,0,14.00,12.00
5.521461,11,27,10,1,0,1,11.00,12.00
6.591674,12,34,16,1,0,1,8.00,7.00
6.214608,12,26,8,1,0,1,9.00,12.00


__(a)__ Use OLS to estimate the parameters of the model

$$ \text{logw} = \beta_1 + \beta_2\text{educ} + \beta_3\text{exper} + \beta_4\text{exper}^2 + \beta_5\text{smsa} + \beta_6\text{south} + \epsilon $$

Give an interpretation to the estimated $\beta_2$ coefficient.

In [4]:
# add the square of "exper" to the dataframe
retsed <- retsed %>% mutate(exper2 = exper^2)
head(retsed)

logw,educ,age,exper,smsa,south,nearc,daded,momed,exper2
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
6.306275,7,29,16,1,0,0,9.94,10.25,256
6.175867,12,27,9,1,0,0,8.00,8.00,81
6.580639,12,34,16,1,0,0,14.00,12.00,256
5.521461,11,27,10,1,0,1,11.00,12.00,100
6.591674,12,34,16,1,0,1,8.00,7.00,256
6.214608,12,26,8,1,0,1,9.00,12.00,64


In [5]:
# regress "logw" on the above variables
ols1 <- lm(logw ~ educ + exper + exper2 + smsa + south, data=retsed)
summary(ols1)


Call:
lm(formula = logw ~ educ + exper + exper2 + smsa + south, data = retsed)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.71487 -0.22987  0.02268  0.24898  1.38552 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  4.6110144  0.0678950  67.914  < 2e-16 ***
educ         0.0815797  0.0034990  23.315  < 2e-16 ***
exper        0.0838357  0.0067735  12.377  < 2e-16 ***
exper2      -0.0022021  0.0003238  -6.800 1.26e-11 ***
smsa         0.1508006  0.0158360   9.523  < 2e-16 ***
south       -0.1751761  0.0146486 -11.959  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.3813 on 3004 degrees of freedom
Multiple R-squared:  0.2632,	Adjusted R-squared:  0.2619 
F-statistic: 214.6 on 5 and 3004 DF,  p-value: < 2.2e-16


The estimated $\beta_2 = 0.08$ coefficient represents the percentage change in wage of an individual for each additional year of schooling, all other variables constant.

__(b)__ OLS may be inconsistent in this case as `educ` and `exper` may be endogenous. Give a reason why this may be the case. Also indicate whether the estimate in part (a) is still useful

That `educ` and `exper` may be endogeneous variables means that they may be affected by an ommitted third variable, which also affects `logwage`. In addition to this, there could be strategic behaviour from the part of workers, which are influenced by the higher wages to, for example, stay more years in school.

If there is in fact endogeneity, the OLS estimate in part (a) will not be consistent, i.e. it will not converge to the true value of `logwage` when $n \to \infty$.

__(c)__ Give a motivation why $\text{age}$ and $\text{age}^2$ can be used as instruments for $\text{exper}$ and $\text{exper}^2$.

For the variables $\text{age}$ and $\text{age}^2$ to be used as instruments, they need to have two characteristics: 

1. They have to be correlated with $\text{exper}$ and $\text{exper}^2$. In this case, it makes sense intuitively that an older worker will have more experience than a younger one, everything else equal.
2. They have to not be correlated with $\text{logwage}$. In the case of age, there may be instances of older workers earning higher wages in more senior positions, but this may not be the case always.

__(d)__ Run the first-stage regression for $\text{educ}$ for the two-stage least squares estimation of the parameters in the model above when $\text{age}$, $\text{age}^2$ , $\text{nearc}$, $\text{dadeduc}$, and $\text{momeduc}$ are used as additional instruments. What do you conclude about the suitability of these instruments for schooling?

In [6]:
# add the square of age to the tribble
retsed <- retsed %>% mutate(age2 = age^2)

# regress "educ" on the instruments "age", "age2", "nearc", "daded",
# and "momed", and the exogenous variables
educ1 <- lm(educ ~ age + age2 + nearc + daded + momed, data=retsed)
summary(educ1)


Call:
lm(formula = educ ~ age + age2 + nearc + daded + momed, data = retsed)

Residuals:
     Min       1Q   Median       3Q      Max 
-11.4573  -1.4968  -0.2734   1.6843   7.5636 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -5.923273   4.010502  -1.477 0.139796    
age          0.992550   0.281060   3.531 0.000419 ***
age2        -0.017075   0.004878  -3.500 0.000472 ***
nearc        0.528751   0.092698   5.704 1.28e-08 ***
daded        0.202048   0.015665  12.898  < 2e-16 ***
momed        0.248379   0.017036  14.580  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 2.346 on 3004 degrees of freedom
Multiple R-squared:  0.233,	Adjusted R-squared:  0.2317 
F-statistic: 182.5 on 5 and 3004 DF,  p-value: < 2.2e-16


In [21]:
# test correlation between "educ" and the exogenous variables
# by performing the F-test on the intruments (if the coefficient of the
# instruments = 0, then the fitted values of "educ" would be almost perfectly
# correlated with the exogenous variables)
educ1_F <- linearHypothesis(educ1, c("nearc=0", "daded=0", "momed=0"))
educ1_F[2,]

,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,3004,16538.05,3,4947.122,299.5346,4.105402e-170


As we can see from the result above, he P-value for the alternative hypothesis is extremely small, which suggests strong evidence agains the null. This means that the instruments are very likely to not be correlated with the potentially endogenous variable $\text{educ}$

__(e)__ Estimate the parameters of the model for log wage using two-stage least squares where you correct for the endogeneity of education __and__ experience. Compare your result to the estimate in part (a).

In [24]:
# perform the 2SLS regression to get the parameters
# of the model, using ivreg(), which automatically
# performs the two stages, first regressing the endogenous
# variables on the instruments, and then regressing "logw"
# on their fitted values
twosls1 <- ivreg(
    logw ~ educ + exper + exper2 + smsa + south | # 2sls equation, same as (a)
    smsa + south + age + age2 + nearc + daded + momed, # include exogenous variables and instruments
    data = retsed
)

summary(twosls1)


Call:
ivreg(formula = logw ~ educ + exper + exper2 + smsa + south | 
    smsa + south + age + age2 + nearc + daded + momed, data = retsed)

Residuals:
    Min      1Q  Median      3Q     Max 
-1.7494 -0.2360  0.0266  0.2498  1.3468 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  4.4169039  0.1154208  38.268  < 2e-16 ***
educ         0.0998429  0.0065738  15.188  < 2e-16 ***
exper        0.0728669  0.0167134   4.360 1.35e-05 ***
exper2      -0.0016393  0.0008381  -1.956   0.0506 .  
smsa         0.1349370  0.0167695   8.047 1.21e-15 ***
south       -0.1589869  0.0156854 -10.136  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.3844 on 3004 degrees of freedom
Multiple R-Squared: 0.2512,	Adjusted R-squared: 0.2499 
Wald test: 175.9 on 5 and 3004 DF,  p-value: < 2.2e-16 


After performing the 2SLS regression, this is the model:

$$ \text{logw} = 4.42 + 0.1\text{educ} + 0.07\text{exper} - 0.00\text{exper}^2 + 0.13\text{smsa} - 0.16\text{south} $$

We can compare this regression to the first one, performed in part (a):

$$ \text{logw} = 4.61 + 0.08\text{educ} + 0.08\text{exper} - 0.00\text{exper}^2 + 0.15\text{smsa} - 0.18\text{south} $$

As we can see, the model in (a) gave higher coefficients to all variables, except $\text{educ}$.

__(f)__ Perform the Sargan test for validity of the instruments. What is your conclusion?

In [38]:
# use diagnostics=TRUE to see the results of the Sargan
# test, i.e. test the null hypothesis that the instruments
# are valid (uncorrelated with the error term)
summary(twosls1, diagnostics = TRUE)$diagnostics["Sargan", ]

df1       df2 statistic   p-value 
2.0000000        NA 3.7023886 0.1570495

From the fact that the $\text{p-value of the Sargan test} = 0.16 > 0.05$, we fail to reject the null hypothesis (all instruments are valid), and we can conclude that the instruments $\text{age}, \text{age}^2, \text{nearc}, \text{dadeduc}$, and $\text{momeduc}$ are likely valid.